In [1]:
%%capture
!pip install -r requirement.txt;

In [2]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
import timeout_decorator
from pulp import *
import time # note: pulp alsp has time function


pd.set_option('display.max_columns', 500) # show all columns
INTINF = 100000 # int inf

debug = 1 # debug mode

# randomly choose n samples
n = 30000

### PARAMETER INPUT

In [3]:
# boolen
NbvCost = False # True: max nbv; False: min cost

# float
minTotalNbv = 5000
maxTotalNbv = 100000000

# float
minTotalCost = 5000
maxTotalCost = 100000000

# float
fleetAgeLowBound = [-INTINF, 3, 4]
fleetAgeUpBound = [3, 6, 10]
fleetAgeLimit = [0.1, 0.1, 0.1]

# float
OnHireLimit = 0.8

# float
weightedAgeLowBound = [-INTINF, 3, 4]
weightedAgeUpBound = [4, 6, 15]
weightedAgeLimit = [0.1, 0.1, 0.1]

# string
productType = ['D40', 'D20', None]
# float
productLimit = [0.1, 0.1, None]

# string
lesseeType = ['MSC', None, None]
# float
lesseeLimit = [0.2, None, None]

# string
contractType = [None, None, None]
# float
contractLimit = [None, None, None]

### READ DATA
TODO: read data from GP

In [4]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')
rawData = rawData.iloc[:30000, :61]

In [5]:
data = rawData.sample(n)
print(data.shape)
data.sample(3)

(30000, 61)


,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,Owner,Teu Fz,Ceu Fz,Contract Cust Id,Contract Num,Contract Dt,Contract Expiration Dt,Contract Lease Type,Contract Currency,Exchange Rate,Per Diem Rate (Max),Per Diem Rate (Max) in USD,Contract Rate Type (Min),Per Diem Rate (Min),Per Diem Rate (Min) in USD,Buy Out Amt Fz,Monthly Depr,Depre adjustment,Nbv,Life Status Fz,Billing Status Fz,Move Status Fz,Move Type Fz,Inv Status Fz,Fleet Year Fz,Financing Cd,Asset Status,Lessor,Remaining Lease Term,Age x CEU,RML x CEU,Mfr Year,Reserve Code Fz,Remaining Lease Term(C),RML x CEU(C),Chk on RML validity(HAPAG),Expiry year,Expiry month,Chk on RML validity(12 YR),Planned,Entity,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units
21258,FZOPS,DF3RM,FDCU0597690,3849.0,CIMC-FL-1804,4/6/2018,1210617.0,DFC3,D4H,FDCU0597690,Dong Fang,2.0,1.7,ONE,LE-ONE-01,2018-02-01,2026-10-14,LE,USD,1.0,1.17,1.17,FA,1.17,1.17,0.0,-16.39,0.663,3123.456,AC,ON,GO,RLI,AVA,4.23,NaN,I,FUSBI,4.295890,7.191,7.303014,2018.0,F - OL / < 8 yrs,4.295890,7.303014,0.0,2026.0,10.0,8.525890,CNIC,FCI,1.7,2018 NP,Reasonable,8.467164,6.026849,from available portfolio,LE-ONE-01D4H,NaN,2539.0,1.0
9779,FZOPS,DF3RM,FFAU3675607,4420.0,DFIC-FL-2104,3/24/2021,1700107.0,DFC3,D4H,FFAU3675607,Dong Fang,2.0,1.7,ONE,LE-ONE-09A,2020-11-16,2029-05-15,LT,USD,1.0,1.25,1.25,ST,0.63,0.63,0.0,-18.91,0.000,4136.340,AC,ON,GO,RLI,AVA,1.27,NaN,I,FUSBI,6.882192,2.159,11.699726,2021.0,F - OL / < 8 yrs,6.882192,11.699726,0.0,2029.0,5.0,8.152192,FCI ANZ 100M,FCI,1.7,2021 NP,Reasonable,3.435164,10.423562,from ANZ portfolio,LE-ONE-09AD4H,NaN,2000.0,1.0
25519,FZOPS,DF3RM,FBIU0319207,2262.0,CIMC-FL-1804,3/21/2018,1182266.0,DFC3,D20,FBIU0319207,Dong Fang,1.0,1.0,ONE,LE-ONE-01,2018-02-01,2026-10-14,LE,USD,1.0,0.69,0.69,FA,0.69,0.69,0.0,-8.23,0.604,1900.275,AC,ON,GO,RLI,AVA,4.27,NaN,I,FUSBI,4.295890,4.270,4.295890,2018.0,F - OL / < 8 yrs,4.295890,4.295890,0.0,2026.0,10.0,8.565890,CNIC,FCI,1.0,2018 NP,Reasonable,5.020685,3.545205,from available portfolio,LE-ONE-01D20,NaN,4991.0,1.0


### DATA PREPARATION

#### Container Age

Assign to `FleetAge1`, `FleetAge2`, `FleetAge3`

In [6]:
def SelectFleetAge(age, i):
    # check input
    if fleetAgeLowBound[i] is None:
        fleetAgeLowBound[i] = -float('inf')
    if fleetAgeUpBound[i] is None:
        fleetAgeUpBound[i] = float('inf')
        
    if fleetAgeLowBound[i] <= age < fleetAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

data['FleetAge1'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 0), axis=1)
data['FleetAge2'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 1), axis=1)
data['FleetAge3'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 2), axis=1)

#### Status

Assign new column `Status`

In [7]:
def SelectStatus(status):
    if status == 'ON':
        status = 1
    else:
        status = 0
    return status

data['Status'] = data.apply(lambda x: SelectStatus(x['Billing Status Fz']), axis=1)

#### Weighted Age

Assign new column `WeightedAge1`, `WeightedAge2`, `WeightedAge3`

In [8]:
def SelectWeightedAge(age, i):
    # check input
    if weightedAgeLowBound[i] is None:
        weightedAgeLowBound[i] = -float('inf')
    if weightedAgeUpBound[i] is None:
        weightedAgeUpBound[i] = float('inf')
        
    if weightedAgeLowBound[i] <= age < weightedAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

data['WeightedAge1'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 0), axis=1)
data['WeightedAge2'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 1), axis=1)
data['WeightedAge3'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 2), axis=1)

#### Product

Assign new column `ProductType1`, `ProductType2`, `ProductType3`

In [9]:
def SelectProductType(product, i):
    if product == productType[i]:
        product = 1
    else:
        product = 0
    return product

data['ProductType1'] = data.apply(lambda x: SelectProductType(x['Product'], 0), axis=1)
data['ProductType2'] = data.apply(lambda x: SelectProductType(x['Product'], 1), axis=1)
data['ProductType3'] = data.apply(lambda x: SelectProductType(x['Product'], 2), axis=1)

#### Lessee

Assign new column `Lessee1`, `Lessee2`, `Lessee3`

In [10]:
def SelectLessee(lessee, i):
    if lessee == lesseeType[i]:
        lessee = 1
    else:
        lessee = 0
    return lessee

data['Lessee1'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 0), axis=1)
data['Lessee2'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 1), axis=1)
data['Lessee3'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 2), axis=1)

#### Contract Type

Assign new column `ContractType1`, `ContractType2`, `ContractType3`

In [11]:
def SelectContractType(contract, i):
    if contract == contractType[i]:
        contract = 1
    else:
        contract = 0
    return contract

data['ContractType1'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 0), axis=1)
data['ContractType2'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 1), axis=1)
data['ContractType3'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 2), axis=1)

### SAVE DATA

In [12]:
if debug:
    data.to_csv('prepared_data_demo.csv')

In [13]:
if debug:
    data = pd.read_csv('./prepared_data_demo.csv')
data.sample(3)

,Unnamed: 0,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,Owner,Teu Fz,Ceu Fz,Contract Cust Id,Contract Num,Contract Dt,Contract Expiration Dt,Contract Lease Type,Contract Currency,Exchange Rate,Per Diem Rate (Max),Per Diem Rate (Max) in USD,Contract Rate Type (Min),Per Diem Rate (Min),Per Diem Rate (Min) in USD,Buy Out Amt Fz,Monthly Depr,Depre adjustment,Nbv,Life Status Fz,Billing Status Fz,Move Status Fz,Move Type Fz,Inv Status Fz,Fleet Year Fz,Financing Cd,Asset Status,Lessor,Remaining Lease Term,Age x CEU,RML x CEU,Mfr Year,Reserve Code Fz,Remaining Lease Term(C),RML x CEU(C),Chk on RML validity(HAPAG),Expiry year,Expiry month,Chk on RML validity(12 YR),Planned,Entity,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units,FleetAge1,FleetAge2,FleetAge3,Status,WeightedAge1,WeightedAge2,WeightedAge3,ProductType1,ProductType2,ProductType3,Lessee1,Lessee2,Lessee3,ContractType1,ContractType2,ContractType3
6637,26109,FZOPS,DF3RM,FBIU0318366,2262.0,CIMC-FL-1804,3/21/2018,1182182.0,DFC3,D20,FBIU0318366,Dong Fang,1.0,1.0,ONE,LE-ONE-01,2018-02-01,2026-10-14,LE,USD,1.0,0.69,0.69,FA,0.69,0.69,0.0,-8.23,0.610,1884.210,AC,ON,GO,RLI,AVA,4.27,NaN,I,FUSBI,4.295890,4.270,4.295890,2018.0,F - OL / < 8 yrs,4.295890,4.295890,0.0,2026.0,10.0,8.565890,CNIC,FCI,1.0,2018 NP,Reasonable,5.020685,3.545205,from available portfolio,LE-ONE-01D20,NaN,4991.0,1.0,0,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0
23385,22011,FZOPS,DF3RM,FBIU0349665,2267.0,CIMC-FL-1804,3/22/2018,1185312.0,DFC3,D20,FBIU0349665,Dong Fang,1.0,1.0,ONE,LE-ONE-01,2018-02-01,2026-10-14,LE,USD,1.0,0.69,0.69,FA,0.69,0.69,0.0,-8.26,0.611,1888.004,AC,ON,GO,RLI,AVA,4.27,NaN,I,FUSBI,4.295890,4.270,4.295890,2018.0,F - OL / < 8 yrs,4.295890,4.295890,0.0,2026.0,10.0,8.565890,CNIC,FCI,1.0,2018 NP,Reasonable,5.020685,3.545205,from available portfolio,LE-ONE-01D20,NaN,4991.0,1.0,0,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0
18801,9452,FZOPS,DF3RM,FFAU3954509,5150.0,DFIC-FL-2107,4/20/2021,1718997.0,DFC3,D4H,FFAU3954509,Dong Fang,2.0,1.7,MSC,LT-MSC-48,2021-04-01,2029-06-30,LT,USD,1.0,1.95,1.95,FR,1.95,1.95,720.0,-22.78,0.005,4773.960,AC,ON,GO,RLI,AVA,1.19,NaN,I,FUSBI,7.008219,2.023,11.913973,2021.0,F - FL / < 8 yrs,7.008219,11.913973,0.0,2029.0,6.0,8.198219,FCI ANZ 100M,FCI,1.7,2021 NP,Reasonable,3.299164,10.637808,from ANZ portfolio,LT-MSC-48D4H,NaN,6997.0,1.0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0


### Model Part

In [14]:
nbv = data['Nbv'].to_numpy()
cost = data['Cost'].to_numpy()
fleetAge = np.stack([data['FleetAge1'].to_numpy(),
                     data['FleetAge2'].to_numpy(),
                     data['FleetAge3'].to_numpy()]).reshape(3, n, 1)
status = data['Status'].to_numpy().reshape(n, 1)
weightedAge = np.stack([data['WeightedAge1'].to_numpy(),
                        data['WeightedAge2'].to_numpy(),
                        data['WeightedAge3'].to_numpy()]).reshape(3, n, 1)
productType = np.stack([data['ProductType1'].to_numpy(),
                        data['ProductType2'].to_numpy(),
                        data['ProductType3'].to_numpy()]).reshape(3, n, 1)
lessee = np.stack([data['Lessee1'].to_numpy(),
                   data['Lessee2'].to_numpy(),
                   data['Lessee3'].to_numpy()]).reshape(3, n, 1)
contractType = np.stack([data['ContractType1'].to_numpy(),
                         data['ContractType2'].to_numpy(),
                         data['ContractType3'].to_numpy(),]).reshape(3, n, 1)
if debug:
    print('nbv shape: ', nbv.shape)
    print('cost shape:', cost.shape)
    print('fleetAge shape: ', fleetAge.shape)
    print('status shape:', status.shape)
    print('weightedAge shape:', weightedAge.shape)
    print('productType shape:', productType.shape)
    print('lessee shape:', lessee.shape)
    print('contractType shape:', contractType.shape)

nbv shape:  (30000,)
cost shape: (30000,)
fleetAge shape:  (3, 30000, 1)
status shape: (30000, 1)
weightedAge shape: (3, 30000, 1)
productType shape: (3, 30000, 1)
lessee shape: (3, 30000, 1)
contractType shape: (3, 30000, 1)


#### Constraints

`x`: np.array, size=num_of_container. `x[i]=1` stands for $i^{th}$ container is selected; `x[i]=0` stands for $i^{th}$ container is not selected.

1. minTotalNbv <= Total NBV <= maxTotalNbv
    
2. minTotalCost <= Total Cost <= maxTotalCost
    
3. Container Age >=
    
4. Status:

    1. OnHire >= OnHireLimit

5. Weighted Age >=

6. Product >=
    
7. Lessee <=
    
8. Contract Type >= 


#### Objective

1. min Cost: `Vars @ cost`

2. max Nbv: `Vars @ nbv`

#### Define Problem
Define objective function and constraint functions

#### Output

In [15]:
containers=[i for i in range(n)]

In [16]:
prob = LpProblem("problem1",LpMaximize)

In [17]:
var = LpVariable.dicts("Container", containers, lowBound=0, cat=LpBinary)

In [18]:
# objective function -- max nbv
prob += lpSum([var[i] * nbv[i] for i in range(n)])

In [19]:
# constraints
prob += lpSum([var[i] * nbv[i] for i in range(n)]) <= 100000000
prob += lpSum([var[i] * nbv[i] for i in range(n)]) >= 10000000
# prob += lpSum([var[i] * cost[i] for i in range(n)]) <= 100000000
# prob += lpSum([var[i] * cost[i] for i in range(n)]) >= 10000000

# prob.writeLP('problem.lp')

In [20]:
# solve
t_start = time.time()
prob.solve()
print('time consumed: ', time.time() - t_start, 's')

time consumed:  102.64084506034851 s


In [ ]:
print("==============================================================")
# print(prob)
print("\n","status:",LpStatus[prob.status],"\n")
print("==============================================================")
for v in prob.variables():
    print("\t",v.name,"=",v.varValue,"\n")
print("target is: ",value(prob.objective))


 status: Optimal 

	 Container_0 = 1.0 

	 Container_1 = 1.0 

	 Container_10 = 1.0 

	 Container_100 = 1.0 

	 Container_1000 = 1.0 

	 Container_10000 = 1.0 

	 Container_10001 = 1.0 

	 Container_10002 = 1.0 

	 Container_10003 = 1.0 

	 Container_10004 = 1.0 

	 Container_10005 = 1.0 

	 Container_10006 = 1.0 

	 Container_10007 = 1.0 

	 Container_10008 = 1.0 

	 Container_10009 = 1.0 

	 Container_1001 = 1.0 

	 Container_10010 = 1.0 

	 Container_10011 = 1.0 

	 Container_10012 = 1.0 

	 Container_10013 = 1.0 

	 Container_10014 = 1.0 

	 Container_10015 = 1.0 

	 Container_10016 = 1.0 

	 Container_10017 = 1.0 

	 Container_10018 = 1.0 

	 Container_10019 = 1.0 

	 Container_1002 = 1.0 

	 Container_10020 = 1.0 

	 Container_10021 = 1.0 

	 Container_10022 = 1.0 

	 Container_10023 = 1.0 

	 Container_10024 = 1.0 

	 Container_10025 = 1.0 

	 Container_10026 = 1.0 

	 Container_10027 = 1.0 

	 Container_10028 = 1.0 

	 Container_10029 = 1.0 

	 Container_1003 = 1.0 

	 Contain

	 Container_10341 = 1.0 

	 Container_10342 = 1.0 

	 Container_10343 = 1.0 

	 Container_10344 = 1.0 

	 Container_10345 = 1.0 

	 Container_10346 = 0.0 

	 Container_10347 = 1.0 

	 Container_10348 = 1.0 

	 Container_10349 = 1.0 

	 Container_1035 = 1.0 

	 Container_10350 = 1.0 

	 Container_10351 = 1.0 

	 Container_10352 = 1.0 

	 Container_10353 = 1.0 

	 Container_10354 = 1.0 

	 Container_10355 = 1.0 

	 Container_10356 = 1.0 

	 Container_10357 = 1.0 

	 Container_10358 = 1.0 

	 Container_10359 = 1.0 

	 Container_1036 = 1.0 

	 Container_10360 = 1.0 

	 Container_10361 = 1.0 

	 Container_10362 = 1.0 

	 Container_10363 = 1.0 

	 Container_10364 = 1.0 

	 Container_10365 = 0.0 

	 Container_10366 = 1.0 

	 Container_10367 = 1.0 

	 Container_10368 = 1.0 

	 Container_10369 = 1.0 

	 Container_1037 = 1.0 

	 Container_10370 = 1.0 

	 Container_10371 = 1.0 

	 Container_10372 = 0.0 

	 Container_10373 = 0.0 

	 Container_10374 = 1.0 

	 Container_10375 = 1.0 

	 Container_103

	 Container_10698 = 1.0 

	 Container_10699 = 1.0 

	 Container_107 = 1.0 

	 Container_1070 = 1.0 

	 Container_10700 = 1.0 

	 Container_10701 = 1.0 

	 Container_10702 = 1.0 

	 Container_10703 = 1.0 

	 Container_10704 = 1.0 

	 Container_10705 = 1.0 

	 Container_10706 = 1.0 

	 Container_10707 = 1.0 

	 Container_10708 = 1.0 

	 Container_10709 = 1.0 

	 Container_1071 = 1.0 

	 Container_10710 = 1.0 

	 Container_10711 = 1.0 

	 Container_10712 = 1.0 

	 Container_10713 = 1.0 

	 Container_10714 = 1.0 

	 Container_10715 = 1.0 

	 Container_10716 = 1.0 

	 Container_10717 = 1.0 

	 Container_10718 = 1.0 

	 Container_10719 = 1.0 

	 Container_1072 = 1.0 

	 Container_10720 = 1.0 

	 Container_10721 = 1.0 

	 Container_10722 = 1.0 

	 Container_10723 = 1.0 

	 Container_10724 = 1.0 

	 Container_10725 = 1.0 

	 Container_10726 = 1.0 

	 Container_10727 = 1.0 

	 Container_10728 = 1.0 

	 Container_10729 = 1.0 

	 Container_1073 = 1.0 

	 Container_10730 = 1.0 

	 Container_10731 

	 Container_11130 = 1.0 

	 Container_11131 = 1.0 

	 Container_11132 = 1.0 

	 Container_11133 = 1.0 

	 Container_11134 = 1.0 

	 Container_11135 = 1.0 

	 Container_11136 = 1.0 

	 Container_11137 = 1.0 

	 Container_11138 = 1.0 

	 Container_11139 = 1.0 

	 Container_1114 = 1.0 

	 Container_11140 = 1.0 

	 Container_11141 = 1.0 

	 Container_11142 = 1.0 

	 Container_11143 = 1.0 

	 Container_11144 = 1.0 

	 Container_11145 = 1.0 

	 Container_11146 = 1.0 

	 Container_11147 = 0.0 

	 Container_11148 = 1.0 

	 Container_11149 = 1.0 

	 Container_1115 = 1.0 

	 Container_11150 = 1.0 

	 Container_11151 = 1.0 

	 Container_11152 = 1.0 

	 Container_11153 = 1.0 

	 Container_11154 = 1.0 

	 Container_11155 = 1.0 

	 Container_11156 = 1.0 

	 Container_11157 = 1.0 

	 Container_11158 = 1.0 

	 Container_11159 = 1.0 

	 Container_1116 = 1.0 

	 Container_11160 = 1.0 

	 Container_11161 = 1.0 

	 Container_11162 = 1.0 

	 Container_11163 = 1.0 

	 Container_11164 = 1.0 

	 Container_111

	 Container_11479 = 0.0 

	 Container_1148 = 1.0 

	 Container_11480 = 0.0 

	 Container_11481 = 1.0 

	 Container_11482 = 1.0 

	 Container_11483 = 1.0 

	 Container_11484 = 1.0 

	 Container_11485 = 1.0 

	 Container_11486 = 1.0 

	 Container_11487 = 1.0 

	 Container_11488 = 1.0 

	 Container_11489 = 1.0 

	 Container_1149 = 0.0 

	 Container_11490 = 1.0 

	 Container_11491 = 1.0 

	 Container_11492 = 1.0 

	 Container_11493 = 1.0 

	 Container_11494 = 1.0 

	 Container_11495 = 1.0 

	 Container_11496 = 1.0 

	 Container_11497 = 1.0 

	 Container_11498 = 1.0 

	 Container_11499 = 1.0 

	 Container_115 = 1.0 

	 Container_1150 = 1.0 

	 Container_11500 = 1.0 

	 Container_11501 = 1.0 

	 Container_11502 = 1.0 

	 Container_11503 = 1.0 

	 Container_11504 = 1.0 

	 Container_11505 = 1.0 

	 Container_11506 = 0.0 

	 Container_11507 = 1.0 

	 Container_11508 = 1.0 

	 Container_11509 = 1.0 

	 Container_1151 = 1.0 

	 Container_11510 = 1.0 

	 Container_11511 = 1.0 

	 Container_11512 

	 Container_11862 = 1.0 

	 Container_11863 = 1.0 

	 Container_11864 = 1.0 

	 Container_11865 = 1.0 

	 Container_11866 = 1.0 

	 Container_11867 = 1.0 

	 Container_11868 = 0.0 

	 Container_11869 = 0.0 

	 Container_1187 = 1.0 

	 Container_11870 = 1.0 

	 Container_11871 = 1.0 

	 Container_11872 = 1.0 

	 Container_11873 = 1.0 

	 Container_11874 = 1.0 

	 Container_11875 = 1.0 

	 Container_11876 = 1.0 

	 Container_11877 = 1.0 

	 Container_11878 = 1.0 

	 Container_11879 = 1.0 

	 Container_1188 = 0.0 

	 Container_11880 = 1.0 

	 Container_11881 = 1.0 

	 Container_11882 = 1.0 

	 Container_11883 = 1.0 

	 Container_11884 = 1.0 

	 Container_11885 = 1.0 

	 Container_11886 = 1.0 

	 Container_11887 = 1.0 

	 Container_11888 = 1.0 

	 Container_11889 = 1.0 

	 Container_1189 = 1.0 

	 Container_11890 = 1.0 

	 Container_11891 = 1.0 

	 Container_11892 = 1.0 

	 Container_11893 = 1.0 

	 Container_11894 = 1.0 

	 Container_11895 = 1.0 

	 Container_11896 = 1.0 

	 Container_118

	 Container_12256 = 1.0 

	 Container_12257 = 1.0 

	 Container_12258 = 1.0 

	 Container_12259 = 1.0 

	 Container_1226 = 1.0 

	 Container_12260 = 1.0 

	 Container_12261 = 1.0 

	 Container_12262 = 1.0 

	 Container_12263 = 1.0 

	 Container_12264 = 1.0 

	 Container_12265 = 1.0 

	 Container_12266 = 1.0 

	 Container_12267 = 1.0 

	 Container_12268 = 1.0 

	 Container_12269 = 1.0 

	 Container_1227 = 1.0 

	 Container_12270 = 1.0 

	 Container_12271 = 1.0 

	 Container_12272 = 1.0 

	 Container_12273 = 1.0 

	 Container_12274 = 0.0 

	 Container_12275 = 1.0 

	 Container_12276 = 1.0 

	 Container_12277 = 1.0 

	 Container_12278 = 1.0 

	 Container_12279 = 1.0 

	 Container_1228 = 0.0 

	 Container_12280 = 1.0 

	 Container_12281 = 1.0 

	 Container_12282 = 1.0 

	 Container_12283 = 1.0 

	 Container_12284 = 1.0 

	 Container_12285 = 1.0 

	 Container_12286 = 1.0 

	 Container_12287 = 1.0 

	 Container_12288 = 1.0 

	 Container_12289 = 1.0 

	 Container_1229 = 1.0 

	 Container_1229


	 Container_12555 = 1.0 

	 Container_12556 = 1.0 

	 Container_12557 = 1.0 

	 Container_12558 = 1.0 

	 Container_12559 = 1.0 

	 Container_1256 = 1.0 

	 Container_12560 = 1.0 

	 Container_12561 = 1.0 

	 Container_12562 = 1.0 

	 Container_12563 = 1.0 

	 Container_12564 = 1.0 

	 Container_12565 = 1.0 

	 Container_12566 = 1.0 

	 Container_12567 = 1.0 

	 Container_12568 = 1.0 

	 Container_12569 = 1.0 

	 Container_1257 = 1.0 

	 Container_12570 = 1.0 

	 Container_12571 = 1.0 

	 Container_12572 = 1.0 

	 Container_12573 = 1.0 

	 Container_12574 = 1.0 

	 Container_12575 = 1.0 

	 Container_12576 = 1.0 

	 Container_12577 = 1.0 

	 Container_12578 = 1.0 

	 Container_12579 = 1.0 

	 Container_1258 = 1.0 

	 Container_12580 = 1.0 

	 Container_12581 = 1.0 

	 Container_12582 = 0.0 

	 Container_12583 = 1.0 

	 Container_12584 = 1.0 

	 Container_12585 = 1.0 

	 Container_12586 = 1.0 

	 Container_12587 = 1.0 

	 Container_12588 = 1.0 

	 Container_12589 = 1.0 

	 Container_12

	 Container_1289 = 1.0 

	 Container_12890 = 1.0 

	 Container_12891 = 1.0 

	 Container_12892 = 1.0 

	 Container_12893 = 1.0 

	 Container_12894 = 1.0 

	 Container_12895 = 1.0 

	 Container_12896 = 1.0 

	 Container_12897 = 0.0 

	 Container_12898 = 1.0 

	 Container_12899 = 1.0 

	 Container_129 = 1.0 

	 Container_1290 = 1.0 

	 Container_12900 = 1.0 

	 Container_12901 = 1.0 

	 Container_12902 = 1.0 

	 Container_12903 = 1.0 

	 Container_12904 = 1.0 

	 Container_12905 = 1.0 

	 Container_12906 = 1.0 

	 Container_12907 = 1.0 

	 Container_12908 = 1.0 

	 Container_12909 = 1.0 

	 Container_1291 = 1.0 

	 Container_12910 = 0.0 

	 Container_12911 = 1.0 

	 Container_12912 = 1.0 

	 Container_12913 = 1.0 

	 Container_12914 = 1.0 

	 Container_12915 = 1.0 

	 Container_12916 = 1.0 

	 Container_12917 = 0.0 

	 Container_12918 = 1.0 

	 Container_12919 = 1.0 

	 Container_1292 = 1.0 

	 Container_12920 = 1.0 

	 Container_12921 = 1.0 

	 Container_12922 = 1.0 

	 Container_12923 